In [49]:
import requests
company = input("Enter company name or ticker: ").strip()


In [50]:

# # -----------------------------
# # CONFIG
# # -----------------------------
# # REPLACE WITH YOUR REAL EMAIL OR SEC WILL BLOCK YOU
# HEADERS = {"User-Agent": "YourName yourname@example.com"}

# def find_cik(company_name: str) -> str:
#     url = "https://www.sec.gov/files/company_tickers.json"
#     res = requests.get(url, headers=HEADERS)
#     res.raise_for_status()
#     data = res.json()

#     for item in data.values():
#         if company_name.lower() in item["title"].lower() or company_name.lower() in item["ticker"].lower():
#             cik = str(item["cik_str"]).zfill(10)
#             return cik
#     raise ValueError(f"Company '{company_name}' not found.")

# def get_financials(cik: str):
#     url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
#     res = requests.get(url, headers=HEADERS)
#     res.raise_for_status()
#     data = res.json()

#     facts = data.get("facts", {}).get("us-gaap", {})

#     def get_latest_fact(tags):
#         """Checks multiple possible tags and returns the most recent USD value."""
#         if isinstance(tags, str):
#             tags = [tags]
            
#         for tag in tags:
#             if tag in facts:
#                 units = facts[tag].get("units", {}).get("USD", [])
#                 if units:
#                     # Sort by 'end' date to ensure we get the most recent period
#                     latest = sorted(units, key=lambda x: x.get('end', ''))[-1]
#                     return {
#                         "val": latest["val"],
#                         "end": latest["end"],
#                         "form": latest.get("form", "N/A")
#                     }
#         return None

#     # Capex is usually found in the Statement of Cash Flows
#     capex_data = get_latest_fact([
#         "PaymentsToAcquirePropertyPlantAndEquipment", 
#         "PaymentsToAcquireProductiveAssets"
#     ])

#     results = {
#         "Revenue": get_latest_fact("Revenues"),
#         "Net Income": get_latest_fact("NetIncomeLoss"),
#         "Total Assets": get_latest_fact("Assets"),
#         "Capex": capex_data
#     }
#     return results

# # -----------------------------
# # Execution
# # -----------------------------
# try:
#     cik = find_cik(company)
#     data = get_financials(cik)

#     print(f"\n--- Financial Data for CIK: {cik} ---")
#     for metric, info in data.items():
#         if info:
#             # Formatting large numbers for readability
#             readable_val = f"${info['val']:,.2f}"
#             print(f"{metric:15} | {readable_val:>20} | Period: {info['end']} ({info['form']})")
#         else:
#             print(f"{metric:15} | Data Not Found")

# except Exception as e:
#     print(f"Error: {e}")

In [51]:
import pandas as pd
import matplotlib.pyplot as plt

In [52]:
# def get_financials(cik: str,metric_name:str):
#     url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
#     res = requests.get(url, headers=HEADERS)
#     res.raise_for_status()
#     data = res.json()
#     if 'facts' in data:
#         # Standard structure: data['facts']['us-gaap'][metric_name]
#         metric_data = None
#         for taxonomy in data['facts'].values():
#             if metric_name in taxonomy:
#                 metric_data = taxonomy[metric_name]
#                 break
#     else:
#         # Snippet structure
#         metric_data = data.get(metric_name)

#     if not metric_data:
#         print(f"Error: Metric '{metric_name}' not found.")
#         return

#     # 3. Extract USD time-series data
#     usd_list = metric_data.get('units', {}).get('USD', [])
#     if not usd_list:
#         print(f"Error: No USD data for {metric_name}.")
#         return

#     # 4. Create DataFrame and convert dates
#     df = pd.DataFrame(usd_list)
#     df['end'] = pd.to_datetime(df['end'])
#     df['filed'] = pd.to_datetime(df['filed'])

#     # 5. DEDUPLICATION LOGIC
#     # SEC data repeats values across filings. We prioritize 10-K over 10-Q 
#     # and always keep the most recently filed version for any given date.
#     form_priority = {'10-K': 1, '10-Q': 2, '8-K': 3}
#     df['priority'] = df['form'].map(lambda x: form_priority.get(x, 4))
    
#     # Sort by date, then form priority, then the date it was filed
#     df = df.sort_values(by=['end', 'priority', 'filed'], ascending=[True, True, False])
    
#     # Keep only one unique data point per period end date
#     df_clean = df.drop_duplicates(subset=['end'], keep='first')

#     # 6. Visualization
#     plt.figure(figsize=(10, 6))
    
#     # Divide by 1 Billion for a cleaner Y-axis
#     plt.plot(df_clean['end'], df_clean['val'] / 1e9, marker='s', color='#2c3e50', linewidth=2)
    
#     plt.title(metric_data.get('label', metric_name), fontsize=12, fontweight='bold')
#     plt.xlabel("Period End Date")
#     plt.ylabel("Billions (USD)")
#     plt.grid(True, linestyle=':', alpha=0.7)
#     plt.xticks(rotation=45)
#     plt.tight_layout()
    
#     # plt.savefig('sec_metric_plot.png')
#     # print(f"Plot generated for {metric_name} and saved as sec_metric_plot.png")

# # --- Usage ---
# # Ensure your data is saved in 'company_facts.json'
# # You can use any metric like 'NetIncomeLoss' or the one provided below

# get_financials(cik,"PaymentsToAcquirePropertyPlantAndEquipment")

In [53]:
# def get_capex_breakdown(cik: str):
#     url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
#     res = requests.get(url, headers=HEADERS)
#     res.raise_for_status()
#     facts = res.json().get("facts", {}).get("us-gaap", {})

#     search_map = {
#         "Total Capex": ["PaymentsToAcquirePropertyPlantAndEquipment", "PaymentsToAcquireProductiveAssets"],
#         "Software/Intangibles": ["PaymentsToAcquireSoftware", "CapitalizedComputerSoftwareCosts"],
#         "Land": ["PaymentsToAcquireLand"]
#     }

#     breakdown = {}
#     for label, tags in search_map.items():
#         for tag in tags:
#             if tag in facts:
#                 # Logic to get 'latest' as you had it
#                 units = facts[tag].get("units", {}).get("USD", [])
#                 if units:
#                     latest = sorted(units, key=lambda x: x.get('end', ''))[-1]
#                     breakdown[label] = {"val": latest["val"], "period": latest["end"]}
#                     break # Stop looking for this label once a tag is found
    
#     return breakdown

# # Usage
# breakdown = get_capex_breakdown(cik) # Apple Example
# breakdown

In [54]:

# -----------------------------
# CONFIG
# -----------------------------
HEADERS = {"User-Agent": "yourname your@email.com"}

# -----------------------------
# Step 1: Find CIK from company name
# -----------------------------
def find_cik(company_name: str) -> str:
    url = "https://www.sec.gov/files/company_tickers.json"
    res = requests.get(url, headers=HEADERS)
    res.raise_for_status()
    data = res.json()

    for item in data.values():
        if company_name.lower() in item["title"].lower():
            cik = str(item["cik_str"]).zfill(10)
            print(f"Found CIK for {item['title']}: {cik}")
            return cik
    raise ValueError(f"Company '{company_name}' not found in SEC tickers.")

# -----------------------------
# Step 2: Get latest filings
# -----------------------------
def get_latest_filing(cik: str, form_types=("10-K","10-Q")) -> dict:
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    res = requests.get(url, headers=HEADERS)
    res.raise_for_status()
    data = res.json()

    for filing in data["filings"]["recent"]["form"]:
        if filing in form_types:
            idx = data["filings"]["recent"]["form"].index(filing)
            accession_number = data["filings"]["recent"]["accessionNumber"][idx].replace("-", "")
            primary_doc = data["filings"]["recent"]["primaryDocument"][idx]
            filing_url = f"https://www.sec.gov/Archives/edgar/data/{int(cik)}/{accession_number}/{primary_doc}"
            return {
                "form": filing,
                "date": data["filings"]["recent"]["filingDate"][idx],
                "url": filing_url
            }
    raise ValueError(f"No {form_types} filing found for CIK {cik}.")

# -----------------------------
# Step 3: Get structured financials (companyfacts API)
# -----------------------------
def get_financials(cik: str):
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    res = requests.get(url, headers=HEADERS)
    res.raise_for_status()
    data = res.json()

    facts = data.get("facts", {}).get("us-gaap", {})

    def get_latest(tag):
        if tag in facts:
            units = facts[tag]["units"]
            if "USD" in units and len(units["USD"]) > 0:
                return units["USD"][-1]["val"]
        return None

    financials = {
        "Revenue": get_latest("Revenues"),
        "Net Income": get_latest("NetIncomeLoss"),
        "Total Assets": get_latest("Assets"),
        "Total Liabilities": get_latest("Liabilities"),
        "Equity": get_latest("StockholdersEquity")
    }
    return financials

# -----------------------------
# Main
# -----------------------------
# if __name__ == "__main__":
# company = input("Enter company name: ").strip()
try:
    cik = find_cik(company)
    latest_filing = get_latest_filing(cik)
    financials = get_financials(cik)

    print("\n=== Latest Filing ===")
    print(f"Form: {latest_filing['form']}")
    print(f"Date: {latest_filing['date']}")
    print(f"Filing URL: {latest_filing['url']}")

    print("\n=== Key Financials (Latest) ===")
    for k, v in financials.items():
        print(f"{k}: {v}")

except Exception as e:
    print("Error:", e)

Found CIK for ASML HOLDING NV: 0000937966
Error: No ('10-K', '10-Q') filing found for CIK 0000937966.


In [55]:
import requests
from bs4 import BeautifulSoup

# -----------------------------
# Config
# -----------------------------
HEADERS = {
    'User-Agent': 'FinancialDataExtractor/1.0 (contact@example.com)'
}

# -----------------------------
# Function to parse XBRL filing
# -----------------------------
def transform_report_from_url(url):
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        return {"error": f"Failed to retrieve URL: {e}"}

    soup = BeautifulSoup(response.content, 'html.parser')
    results = {}

    # 1. Extract non-numeric identifiers (Company Name, CIK, etc.)
    for tag in soup.find_all('ix:nonnumeric'):
        name = tag.get('name')
        if name:
            value = " ".join(tag.get_text().split())
            results[name] = value

    # 2. Extract numeric financial values
    for tag in soup.find_all('ix:nonfraction'):
        name = tag.get('name')
        if not name:
            continue

        raw_text = tag.get_text().strip().replace(',', '')
        if not raw_text or raw_text == '—' or tag.get('xsi:nil') == 'true':
            value = 0.0
        else:
            try:
                value = float(raw_text)
                scale = tag.get('scale')
                if scale:
                    value *= (10 ** int(scale))
                if tag.get('sign') == '-':
                    value = -abs(value)
            except ValueError:
                value = raw_text

        if name in results:
            if not isinstance(results[name], list):
                results[name] = [results[name]]
            results[name].append(value)
        else:
            results[name] = value

    # 3. Extract CAPEX-related data
    capex_tags = [
        "us-gaap:PropertyPlantAndEquipmentAdditions",
        "us-gaap:CapitalExpenditures",
        "us-gaap:PaymentsToAcquirePropertyPlantAndEquipment",
    ]

    capex_data = {}
    for tag_name in capex_tags:
        if tag_name in results:
            capex_data[tag_name] = results[tag_name]

    results['CAPEX'] = capex_data
    return results

# -----------------------------
# Example usage
# -----------------------------
# if __name__ == "__main__":
latest_filing_url = latest_filing['url']

financial_data = transform_report_from_url(latest_filing_url)

if "error" in financial_data:
    print(financial_data["error"])
else:
    # Non-numeric identifiers
    print(f"Registrant: {financial_data.get('dei:EntityRegistrantName')}")
    print(f"CIK: {financial_data.get('dei:EntityCentralIndexKey')}\n")

    # Some key financials

    # CAPEX
    print("=== CAPEX Data ===")
    for k, v in financial_data.get('CAPEX', {}).items():
        print(f"{k}: {v}")

Registrant: VWF Bancorp, Inc.
CIK: 0001913838

=== CAPEX Data ===
us-gaap:PaymentsToAcquirePropertyPlantAndEquipment: [964987.0, 673197.0]


In [56]:
import os
import base64
import requests
from ddgs import DDGS
import anthropic
from functools import lru_cache


QUERIES = [
    "{company} management report 2025 filetype:pdf",
    "{company} annual management report 2025 filetype:pdf",
    "{company} management discussion and analysis 2025 filetype:pdf",
    "{company} MD&A report 2025 filetype:pdf",
    "{company} directors report 2025 filetype:pdf",
    "{company} board report 2025 filetype:pdf",
    "{company} annual report 2025 management section filetype:pdf",
    "{company} executive management report 2025 pdf",
    "{company} leadership report 2025 filetype:pdf",
    "{company} operational report 2025 filetype:pdf",
    "{company} financial review 2025 management report pdf",
    "{company} strategic report 2025 filetype:pdf",
    "{company} corporate report 2025 management pdf",
    "{company} management report site:{company_slug}.com",
    "{company} business review 2025 filetype:pdf",
    "{company} company performance report 2025 pdf",
    "{company} annual performance report 2025 filetype:pdf",
    "{company} consolidated management report 2025 pdf",
    "{company} governance and management report 2025 filetype:pdf",
    "{company} management summary report 2025 pdf",
]


def iter_esg_results(company: str):
    """Generator: yields search results one query at a time, stopping when called to."""
    seen_urls = set()
    slug = company.lower().replace(" ", "")

    for i, template in enumerate(QUERIES):
        query = template.format(company=company, company_slug=slug)
        print(f"[{i+1}/{len(QUERIES)}] Searching: {query}")
        try:
            with DDGS() as ddgs:
                results = list(ddgs.text(query, max_results=5))
            for r in results:
                url = r.get("href", "")
                if url and url not in seen_urls:
                    seen_urls.add(url)
                    yield r
        except Exception as e:
            print(f"  Search failed: {e}")


def try_download_pdf(company: str, url: str) -> str | None:
    """Try to download a single URL as a PDF. Returns file path on success, None otherwise."""
    if not url.lower().endswith(".pdf"):
        return None
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, headers=headers, timeout=15)
        content_type = response.headers.get("Content-Type", "").lower()
        if response.status_code == 200 and "pdf" in content_type:
            os.makedirs("reports", exist_ok=True)
            filename = f"reports/{company.replace(' ', '_')}_ESG_report.pdf"
            with open(filename, "wb") as f:
                f.write(response.content)
            print(f"Downloaded: {filename}\nSource: {url}")
            return filename
    except Exception as e:
        print(f"  Failed to download {url}: {e}")
    return None

@lru_cache(maxsize=10)
def find_and_download_esg_pdf(company: str) -> str | None:
    """Iterate search results and stop as soon as a PDF is successfully downloaded."""
    for result in iter_esg_results(company):
        url = result.get("href", "")
        pdf_path = try_download_pdf(company, url)
        if pdf_path:
            print("PDF found — stopping search early.")
            return pdf_path

    print("Search exhausted. No PDF could be downloaded.")
    return None



In [57]:
find_and_download_esg_pdf(company)

[1/20] Searching: asml management report 2025 filetype:pdf
Downloaded: reports/asml_ESG_report.pdf
Source: https://ourbrand.asml.com/m/39670b708ea03976/original/2025_07_16_Presentation-Investor-Relations-Q2-2025.pdf
PDF found — stopping search early.


'reports/asml_ESG_report.pdf'

In [58]:

# import anthropic
# import base64
# import fitz  # pymupdf
# import os


# def analyze_esg_chunk(client: anthropic.Anthropic, company: str, text: str, page_start: int, page_end: int) -> str:
#     """Send extracted text chunk to Claude for ESG analysis."""
#     response = client.messages.create(
#         model="claude-opus-4-6",
#         max_tokens=4096,
#         messages=[
#             {
#                 "role": "user",
#                 "content": f"""Analyze this section of {company}'s ESG report (pages {page_start}-{page_end}) and split all content into two clear categories:

# ## 1. QUANTIFIABLE DATA
# Extract every concrete, measurable figure mentioned. For each one include:
# - Metric name
# - Value + unit
# - Year / time period
# - Context (what it refers to)

# Examples of what to look for:
# - Financial investments (e.g. datacenter spend, renewable energy investment)
# - Emissions (CO2, Scope 1/2/3 in tonnes)
# - Energy consumption (MWh, GWh)
# - Water usage (litres, m³)
# - Waste (tonnes recycled/landfilled)
# - Workforce stats (headcount, % women in leadership, pay gap %)
# - Certifications, targets with deadlines and % progress
# - Supply chain figures (# suppliers audited, % compliant)

# ## 2. CORPORATE SPEAK
# Flag all vague, unverifiable, or purely qualitative claims. For each one include:
# - The exact phrase or sentence
# - Why it's vague (no metric, no deadline, no baseline, etc.)
# - Suspected intent (reputation management, regulatory compliance, etc.)

# Be thorough and critical. Format your response clearly under these two headings.

# --- REPORT TEXT ---
# {text}"""
#             }
#         ],
#     )
#     return response.content[0].text


# def analyze_esg_pdf(company: str, pdf_path: str) -> str:
#     """Extract text from PDF, chunk it, and analyze each chunk."""
#     print(f"\nOpening PDF: {pdf_path}")

#     client = anthropic.Anthropic()
#     doc = fitz.open(pdf_path)
#     total_pages = len(doc)
#     chunk_size = 50
#     results = []

#     print(f"Total pages: {total_pages}")

#     for start in range(0, total_pages, chunk_size):
#         end = min(start + chunk_size, total_pages)
#         print(f"\nAnalyzing pages {start+1} to {end}...")

#         # Extract text from pages
#         text = ""
#         for page_num in range(start, end):
#             text += f"\n--- Page {page_num+1} ---\n"
#             text += doc[page_num].get_text()

#         if not text.strip():
#             print(f"No text found on pages {start+1}-{end}, skipping...")
#             continue

#         result = analyze_esg_chunk(client, company, text, start + 1, end)
#         results.append(f"## Pages {start+1}-{end}\n\n{result}")

#     doc.close()

#     final_report = f"# ESG Analysis: {company}\n\n" + "\n\n---\n\n".join(results)
#     return final_report

# # if __name__ == "__main__":
# #     company = "YourCompany"
# #     pdf_path = "report.pdf"

# #     report = analyze_esg_pdf(company, pdf_path)
    
# #     # Save to file
# #     output_path = f"{company}_esg_analysis.txt"
# #     with open(output_path, "w", encoding="utf-8") as f:
# #         f.write(report)
    
# #     print(f"\nAnalysis saved to {output_path}")
# def save_analysis(company: str, analysis: str) -> str:
#     """Save Claude's analysis to a text file. Returns the file path."""
#     analysis_path = f"reports/{company.replace(' ', '_')}_ESG_analysis.txt"
#     with open(analysis_path, "w", encoding="utf-8") as f:
#         f.write(f"ESG REPORT ANALYSIS: {company}\n")
#         f.write("=" * 60 + "\n\n")
#         f.write(analysis)
#     print(f"Analysis saved to: {analysis_path}")
#     return analysis_path


# def get_latest_esg_report(company: str) -> dict:
#     """Orchestrator: search (with early termination), download, analyze, and save."""
#     pdf_path = find_and_download_esg_pdf(company)
#     if not pdf_path:
#         return {"company": company, "downloaded": None, "analysis": None, "analysis_path": None}

#     analysis = analyze_esg_pdf(company, pdf_path)
#     analysis_path = save_analysis(company, analysis)

#     return {
#         "company": company,
#         "downloaded": pdf_path,
#         "analysis_path": analysis_path,
#         "analysis": analysis,
#     }


In [59]:
# # --- Run ---
# result = get_latest_esg_report(company)
# print(f"\n{'='*60}")
# print(f"ANALYSIS FOR: {company}")
# print(f"{'='*60}")
# print(result["analysis"])